# Args

In [ ]:
args = {
        'device': 0,
        'num_layer': 3,
        'emb_dim': 256,
        'drop_ratio': 0.5,
        'batch_size': 64,
        'lr': 0.001,
        'epochs': 50,
        'num_vocab': 5000,
        'max_seq_len': 5,
        'diff_pool_layers': [(32, 5)],
        'max_num_nodes': 1000,
        'random_split': False,
        'dataset': "ogbg-code2",
        'num_workers': 0,
        'model_save_path': "best_model_params",
        'eval_results_path': 'eval_results'
    }

# Setting up the dependencies and download the dataset

In [ ]:
!pip install ogb
!pip install torch_geometric
!python -c "import ogb; print(ogb.__version__)"

import os
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import torch
import pandas as pd
import torch.nn.functional as F
from tqdm.notebook import tqdm
print(torch.__version__)

# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from torch_geometric.nn import global_add_pool, global_mean_pool

dataset = PygGraphPropPredDataset(name = "ogbg-code2")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=73a0be2d9be8d4f51f9eeaf4fcde85b5e9f88eb96c861bf724b14bc061391212
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.9 MB/s eta 0:00:00
1.3.6
2.1.0+cu118


Downloaded 0.91 GB: 100%|██████████| 934/934 [00:16<00:00, 57.54it/s]


Extracting dataset/code2.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 452741/452741 [00:01<00:00, 340397.45it/s]


Converting graphs into PyG objects...


100%|██████████| 452741/452741 [00:25<00:00, 17624.56it/s]


Saving...


Done!


# Utils

In [ ]:
from collections import Counter
import numpy as np
import torch

class ASTNodeEncoder(torch.nn.Module):
    '''
        Input:
            x: default node feature. the first and second column represents node type and node attributes.
            depth: The depth of the node in the AST.

        Output:
            emb_dim-dimensional vector

    '''
    def __init__(self, emb_dim, num_nodetypes, num_nodeattributes, max_depth):
        super(ASTNodeEncoder, self).__init__()

        self.max_depth = max_depth

        self.type_encoder = torch.nn.Embedding(num_nodetypes, emb_dim)
        self.attribute_encoder = torch.nn.Embedding(num_nodeattributes, emb_dim)
        self.depth_encoder = torch.nn.Embedding(self.max_depth + 1, emb_dim)


    def forward(self, x, depth):
        depth[depth > self.max_depth] = self.max_depth
        return self.type_encoder(x[:,0]) + self.attribute_encoder(x[:,1]) + self.depth_encoder(depth)



def get_vocab_mapping(seq_list, num_vocab):
    '''
        Input:
            seq_list: a list of sequences
            num_vocab: vocabulary size
        Output:
            vocab2idx:
                A dictionary that maps vocabulary into integer index.
                Additioanlly, we also index '__UNK__' and '__EOS__'
                '__UNK__' : out-of-vocabulary term
                '__EOS__' : end-of-sentence

            idx2vocab:
                A list that maps idx to actual vocabulary.

    '''

    vocab_cnt = {}
    vocab_list = []
    for seq in seq_list:
        for w in seq:
            if w in vocab_cnt:
                vocab_cnt[w] += 1
            else:
                vocab_cnt[w] = 1
                vocab_list.append(w)

    cnt_list = np.array([vocab_cnt[w] for w in vocab_list])
    topvocab = np.argsort(-cnt_list, kind = 'stable')[:num_vocab]

    print('Coverage of top {} vocabulary:'.format(num_vocab))
    print(float(np.sum(cnt_list[topvocab]))/np.sum(cnt_list))

    vocab2idx = {vocab_list[vocab_idx]: idx for idx, vocab_idx in enumerate(topvocab)}
    idx2vocab = [vocab_list[vocab_idx] for vocab_idx in topvocab]

    # print(topvocab)
    # print([vocab_list[v] for v in topvocab[:10]])
    # print([vocab_list[v] for v in topvocab[-10:]])

    vocab2idx['__UNK__'] = num_vocab
    idx2vocab.append('__UNK__')

    vocab2idx['__EOS__'] = num_vocab + 1
    idx2vocab.append('__EOS__')

    # test the correspondence between vocab2idx and idx2vocab
    for idx, vocab in enumerate(idx2vocab):
        assert(idx == vocab2idx[vocab])

    # test that the idx of '__EOS__' is len(idx2vocab) - 1.
    # This fact will be used in decode_arr_to_seq, when finding __EOS__
    assert(vocab2idx['__EOS__'] == len(idx2vocab) - 1)

    return vocab2idx, idx2vocab

def augment_edge(data):
    '''
        Input:
            data: PyG data object
        Output:
            data (edges are augmented in the following ways):
                data.edge_index: Added next-token edge. The inverse edges were also added.
                data.edge_attr (torch.Long):
                    data.edge_attr[:,0]: whether it is AST edge (0) for next-token edge (1)
                    data.edge_attr[:,1]: whether it is original direction (0) or inverse direction (1)
    '''

    ##### AST edge
    edge_index_ast = data.edge_index
    edge_attr_ast = torch.zeros((edge_index_ast.size(1), 2))

    ##### Inverse AST edge
    edge_index_ast_inverse = torch.stack([edge_index_ast[1], edge_index_ast[0]], dim = 0)
    edge_attr_ast_inverse = torch.cat([torch.zeros(edge_index_ast_inverse.size(1), 1), torch.ones(edge_index_ast_inverse.size(1), 1)], dim = 1)


    ##### Next-token edge

    ## Obtain attributed nodes and get their indices in dfs order
    # attributed_node_idx = torch.where(data.node_is_attributed.view(-1,) == 1)[0]
    # attributed_node_idx_in_dfs_order = attributed_node_idx[torch.argsort(data.node_dfs_order[attributed_node_idx].view(-1,))]

    ## Since the nodes are already sorted in dfs ordering in our case, we can just do the following.
    attributed_node_idx_in_dfs_order = torch.where(data.node_is_attributed.view(-1,) == 1)[0]

    ## build next token edge
    # Given: attributed_node_idx_in_dfs_order
    #        [1, 3, 4, 5, 8, 9, 12]
    # Output:
    #    [[1, 3, 4, 5, 8, 9]
    #     [3, 4, 5, 8, 9, 12]
    edge_index_nextoken = torch.stack([attributed_node_idx_in_dfs_order[:-1], attributed_node_idx_in_dfs_order[1:]], dim = 0)
    edge_attr_nextoken = torch.cat([torch.ones(edge_index_nextoken.size(1), 1), torch.zeros(edge_index_nextoken.size(1), 1)], dim = 1)


    ##### Inverse next-token edge
    edge_index_nextoken_inverse = torch.stack([edge_index_nextoken[1], edge_index_nextoken[0]], dim = 0)
    edge_attr_nextoken_inverse = torch.ones((edge_index_nextoken.size(1), 2))


    data.edge_index = torch.cat([edge_index_ast, edge_index_ast_inverse, edge_index_nextoken, edge_index_nextoken_inverse], dim = 1)
    data.edge_attr = torch.cat([edge_attr_ast,   edge_attr_ast_inverse, edge_attr_nextoken,  edge_attr_nextoken_inverse], dim = 0)

    return data

def encode_y_to_arr(data, vocab2idx, max_seq_len):
    '''
    Input:
        data: PyG graph object
        output: add y_arr to data
    '''

    # PyG >= 1.5.0
    seq = data.y

    # PyG = 1.4.3
    # seq = data.y[0]

    data.y_arr = encode_seq_to_arr(seq, vocab2idx, max_seq_len)

    return data

def encode_seq_to_arr(seq, vocab2idx, max_seq_len):
    '''
    Input:
        seq: A list of words
        output: add y_arr (torch.Tensor)
    '''

    augmented_seq = seq[:max_seq_len] + ['__EOS__'] * max(0, max_seq_len - len(seq))
    return torch.tensor([[vocab2idx[w] if w in vocab2idx else vocab2idx['__UNK__'] for w in augmented_seq]], dtype = torch.long)


def decode_arr_to_seq(arr, idx2vocab):
    '''
        Input: torch 1d array: y_arr
        Output: a sequence of words.
    '''


    eos_idx_list = torch.nonzero(arr == len(idx2vocab) - 1, as_tuple=False) # find the position of __EOS__ (the last vocab in idx2vocab)
    if len(eos_idx_list) > 0:
        clippted_arr = arr[: torch.min(eos_idx_list)] # find the smallest __EOS__
    else:
        clippted_arr = arr

    return list(map(lambda x: idx2vocab[x], clippted_arr.cpu()))


# Define GNN and DiffPool

In [ ]:
from torch_geometric.nn import dense_diff_pool, global_add_pool, global_mean_pool, global_max_pool, GlobalAttention, Set2Set, DenseGCNConv
from torch_geometric.utils import to_dense_batch, to_dense_adj
import torch.nn.functional as F

# The generic GCN used by different layers of DiffPool
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
        super(GCN, self).__init__()

        # GCN layers
        self.convs = torch.nn.ModuleList()
        self.convs.extend([DenseGCNConv(hidden_dim, hidden_dim) for i in range(num_layers-1)])
        self.convs.append(DenseGCNConv(hidden_dim, output_dim))

        # Batch norm
        self.bns = torch.nn.ModuleList()
        self.bns.extend([torch.nn.BatchNorm1d(hidden_dim) for i in range(num_layers-1)])

        # Log Softmax
        self.softmax = torch.nn.LogSoftmax(dim=1)

        # Probability of an element getting zeroed
        self.dropout = dropout

    def forward(self, x, adj):
        for i in range(len(self.convs)-1):
          x = self.convs[i](x, adj)
          x = self.bns[i](x.transpose(1,2))
          x = x.transpose(1,2)
          x = F.relu(x)
          x = F.dropout(x, p=self.dropout, training=self.training)
        out = self.convs[-1](x, adj)

        return out

# A DiffPool layer that takes in a graph and pool the nodes to produce a
# smaller graph
class DiffPool(torch.nn.Module):
    def __init__(self, gnn_embed, gnn_pool, number_of_nodes_out):
        super(DiffPool, self).__init__()
        self.gnn_embed = gnn_embed
        self.gnn_pool = gnn_pool
        self.number_of_nodes_out = number_of_nodes_out

    def forward(self, x, adj, mask=None):
        # Compute embeddings and assignment matrix with GNNs.
        z = self.gnn_embed(x, adj)
        s = self.gnn_pool(x, adj)

        # Use the DiffPool to get pooled graph.
        x, adj, l1, e1 = dense_diff_pool(z, adj, s, mask)

        return x, adj



# The top-level abstraction for a composition of networks that takes in a batch
# of data and proudces a prediction
class GNN(torch.nn.Module):
    def __init__(self, num_vocab, max_seq_len, node_encoder, gnn_num_layers = 3, diff_pool_node_number_list=[(50, 3), (10, 3), (3, 3)], emb_dim = 300, drop_ratio = 0.5, graph_pooling = "mean", max_num_nodes=1000):
        super(GNN, self).__init__()
        self.drop_ratio = drop_ratio
        self.emb_dim = emb_dim
        self.num_vocab = num_vocab
        self.max_seq_len = max_seq_len
        self.graph_pooling = graph_pooling
        self.dropout_ratio = drop_ratio
        self.max_num_nodes = max_num_nodes

        if gnn_num_layers < 2:
            raise ValueError("Number of GNN layers must be greater than 1.")


        self.node_encoder = node_encoder

        # Define the DiffPool layers
        self.diff_pool_layers = torch.nn.ModuleList()
        for number_of_nodes_out, gcn_layer_num in diff_pool_node_number_list:
            gnn_embed = GCN(input_dim=emb_dim, hidden_dim=emb_dim, output_dim=emb_dim, num_layers=gcn_layer_num, dropout=drop_ratio)
            gnn_pool = GCN(input_dim=emb_dim, hidden_dim=emb_dim, output_dim=number_of_nodes_out, num_layers=gcn_layer_num, dropout=drop_ratio)
            diff_pool_layer = DiffPool(gnn_embed=gnn_embed, gnn_pool=gnn_pool, number_of_nodes_out=number_of_nodes_out)
            self.diff_pool_layers.append(diff_pool_layer)

        # Define the final GNN that takes the output of the final
        self.final_gnn = GCN(input_dim=emb_dim, hidden_dim=emb_dim, output_dim=emb_dim, num_layers=gnn_num_layers, dropout=drop_ratio)

        # Pooling function to generate whole-graph embeddings
        if self.graph_pooling == "sum":
            self.pool = global_add_pool
        elif self.graph_pooling == "mean":
            self.pool = global_mean_pool
        elif self.graph_pooling == "max":
            self.pool = global_max_pool
        elif self.graph_pooling == "attention":
            self.pool = GlobalAttention(gate_nn = torch.nn.Sequential(torch.nn.Linear(emb_dim, 2*emb_dim), torch.nn.BatchNorm1d(2*emb_dim), torch.nn.ReLU(), torch.nn.Linear(2*emb_dim, 1)))
        elif self.graph_pooling == "set2set":
            self.pool = Set2Set(emb_dim, processing_steps = 2)
        else:
            raise ValueError("Invalid graph pooling type.")

        self.graph_pred_linear_list = torch.nn.ModuleList()

        if graph_pooling == "set2set":
            for i in range(max_seq_len):
                 self.graph_pred_linear_list.append(torch.nn.Linear(2*emb_dim, self.num_vocab))

        else:
            for i in range(max_seq_len):
                 self.graph_pred_linear_list.append(torch.nn.Linear(emb_dim, self.num_vocab))

    def forward(self, batched_data):
        x, edge_index, edge_attr, node_depth, batch = batched_data.x, batched_data.edge_index, batched_data.edge_attr, batched_data.node_depth, batched_data.batch
        # Embed the nodes of the AST
        x = self.node_encoder(x, node_depth.view(-1,))
        # DiffPool Layers
        z, mask = to_dense_batch(x, batch, max_num_nodes=self.max_num_nodes)
        s = to_dense_adj(edge_index, batch, max_num_nodes=self.max_num_nodes)
        for layer in range(len(self.diff_pool_layers)):
            if (mask is not None):
              z, s = self.diff_pool_layers[layer](z, s, mask)
              mask = None
            else:
              z, s = self.diff_pool_layers[layer](z, s)

        # DiffPool Layeres without node_num limit
        # z, mask = to_dense_batch(x, batch)
        # s = to_dense_adj(edge_index, batch)
        # for layer in range(len(self.diff_pool_layers)):
        #     z, s = self.diff_pool_layers[layer](z, s)

        # Final GNN and pooling
        x = self.final_gnn(z, s)
        del z, s, mask
        graph_emb = self.pool(x, batch=None)

        pred_list = []

        for i in range(self.max_seq_len):
            pred_list.append(self.graph_pred_linear_list[i](graph_emb))

        return pred_list




# Main Training Loop

In [ ]:
import numpy as np
import argparse
from torchvision import transforms
import torch.optim as optim

multicls_criterion = torch.nn.CrossEntropyLoss()

def train(model, device, loader, optimizer):
    model.train()

    loss_accum = 0
    iter = 0
    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)
        iter += 1

        if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
            pass
        else:
            optimizer.zero_grad()
            pred_list = model(batch)
            # if (iter % 100 == 1):
            #     print("after pred", iter, " memory use:", torch.cuda.memory_allocated() / 1e6, "MB")

            loss = 0
            for i in range(len(pred_list)):
                loss += multicls_criterion(pred_list[i].to(torch.float32), batch.y_arr[:,i])

            loss = loss / len(pred_list)

            loss.backward()
            optimizer.step()

            # loss_accum += loss.item()
            loss_accum += float(loss)
            del pred_list
            del loss

    print('Average training loss: {}'.format(loss_accum / (step + 1)))


def eval(model, device, loader, evaluator, arr_to_seq):
    model.eval()
    seq_ref_list = []
    seq_pred_list = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred_list = model(batch)

            mat = []
            for i in range(len(pred_list)):
                mat.append(torch.argmax(pred_list[i], dim = 1).view(-1,1))
            mat = torch.cat(mat, dim = 1)

            seq_pred = [arr_to_seq(arr) for arr in mat]

            # PyG = 1.4.3
            # seq_ref = [batch.y[i][0] for i in range(len(batch.y))]

            # PyG >= 1.5.0
            seq_ref = [batch.y[i] for i in range(len(batch.y))]

            seq_ref_list.extend(seq_ref)
            seq_pred_list.extend(seq_pred)

    input_dict = {"seq_ref": seq_ref_list, "seq_pred": seq_pred_list}

    return evaluator.eval(input_dict)

def main():
    print ("Starting Training")
    # Training settings
    print(args)

    device = torch.device("cuda:" + str(args['device'])) if torch.cuda.is_available() else torch.device("cpu")

    # ### automatic dataloading and splitting
    # dataset = PygGraphPropPredDataset(name = args.dataset)

    seq_len_list = np.array([len(seq) for seq in dataset.data.y])
    print('Target seqence less or equal to {} is {}%.'.format(args['max_seq_len'], np.sum(seq_len_list <= args['max_seq_len']) / len(seq_len_list)))

    split_idx = dataset.get_idx_split()

    if args['random_split']:
        print('Using random split')
        perm = torch.randperm(len(dataset))
        num_train, num_valid, num_test = len(split_idx['train']), len(split_idx['valid']), len(split_idx['test'])
        split_idx['train'] = perm[:num_train]
        split_idx['valid'] = perm[num_train:num_train+num_valid]
        split_idx['test'] = perm[num_train+num_valid:]

        assert(len(split_idx['train']) == num_train)
        assert(len(split_idx['valid']) == num_valid)
        assert(len(split_idx['test']) == num_test)


    print(split_idx['train'])
    print(split_idx['valid'])
    print(split_idx['test'])

    print(len(split_idx['train']))
    #shrink the train set
    # split_idx['train'] = split_idx['train'][:len(split_idx['train'])//10]
    # print(len(split_idx['train']))


    ### building vocabulary for sequence predition. Only use training data.

    vocab2idx, idx2vocab = get_vocab_mapping([dataset.data.y[i] for i in split_idx['train']], args['num_vocab'])

    ### set the transform function
    # augment_edge: add next-token edge as well as inverse edges. add edge attributes.
    # encode_y_to_arr: add y_arr to PyG data object, indicating the array representation of a sequence.
    dataset.transform = transforms.Compose([augment_edge, lambda data: encode_y_to_arr(data, vocab2idx, args['max_seq_len'])])

    ### automatic evaluator. takes dataset name as input
    evaluator = Evaluator(args['dataset'])

    train_loader = DataLoader(dataset[split_idx["train"]], batch_size=args['batch_size'], shuffle=True, num_workers = args['num_workers'])
    valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=args['batch_size'], shuffle=False, num_workers = args['num_workers'])
    test_loader = DataLoader(dataset[split_idx["test"]], batch_size=args['batch_size'], shuffle=False, num_workers = args['num_workers'])

    nodetypes_mapping = pd.read_csv(os.path.join(dataset.root, 'mapping', 'typeidx2type.csv.gz'))
    nodeattributes_mapping = pd.read_csv(os.path.join(dataset.root, 'mapping', 'attridx2attr.csv.gz'))

    print(nodeattributes_mapping)

    ### Encoding node features into emb_dim vectors.
    ### The following three node features are used.
    # 1. node type
    # 2. node attribute
    # 3. node depth
    node_encoder = ASTNodeEncoder(args['emb_dim'], num_nodetypes = len(nodetypes_mapping['type']), num_nodeattributes = len(nodeattributes_mapping['attr']), max_depth = 20)

    # if args.gnn == 'gcn':
    model = GNN(num_vocab = len(vocab2idx), max_seq_len = args['max_seq_len'],
                    node_encoder = node_encoder,
                    gnn_num_layers = args['num_layer'],
                    diff_pool_node_number_list = args['diff_pool_layers'],
                    emb_dim = args['emb_dim'],
                    drop_ratio = args['drop_ratio'],
                    graph_pooling = "sum",
                    max_num_nodes=args['max_num_nodes']).to(device)
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    print('model size: {:.3f}MB'.format(size_all_mb))


    # elif args.gnn == 'gin-virtual':
    #     model = GNN(num_vocab = len(vocab2idx), max_seq_len = args.max_seq_len, node_encoder = node_encoder, num_layer = args.num_layer, gnn_type = 'gin', emb_dim = args.emb_dim, drop_ratio = args.drop_ratio, virtual_node = True).to(device)
    # elif args.gnn == 'gcn':
    #     model = GNN(num_vocab = len(vocab2idx), max_seq_len = args.max_seq_len, node_encoder = node_encoder, num_layer = args.num_layer, gnn_type = 'gcn', emb_dim = args.emb_dim, drop_ratio = args.drop_ratio, virtual_node = False).to(device)
    # elif args.gnn == 'gcn-virtual':
    #     model = GNN(num_vocab = len(vocab2idx), max_seq_len = args.max_seq_len, node_encoder = node_encoder, num_layer = args.num_layer, gnn_type = 'gcn', emb_dim = args.emb_dim, drop_ratio = args.drop_ratio, virtual_node = True).to(device)
    # else:
    #     raise ValueError('Invalid GNN type')

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f'#Params: {sum(p.numel() for p in model.parameters())}')

    valid_curve = []
    test_curve = []
    train_curve = []

    best_val_f1 = -1

    for epoch in range(1, args['epochs'] + 1):
        print("=====Epoch {}".format(epoch))
        print('Training...')
        train(model, device, train_loader, optimizer)

        print('Evaluating...')
        # train_perf = eval(model, device, train_loader, evaluator, arr_to_seq = lambda arr: decode_arr_to_seq(arr, idx2vocab))
        valid_perf = eval(model, device, valid_loader, evaluator, arr_to_seq = lambda arr: decode_arr_to_seq(arr, idx2vocab))
        test_perf = eval(model, device, test_loader, evaluator, arr_to_seq = lambda arr: decode_arr_to_seq(arr, idx2vocab))

        # print({'Train': train_perf, 'Validation': valid_perf, 'Test': test_perf})
        print({'Validation': valid_perf, 'Test': test_perf})
        val_f1 = valid_perf[dataset.eval_metric]
        if (val_f1 > best_val_f1):
            best_val_f1 = val_f1
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, args['model_save_path'])

        # train_curve.append(train_perf[dataset.eval_metric])
        valid_curve.append(valid_perf[dataset.eval_metric])
        test_curve.append(test_perf[dataset.eval_metric])

    print('F1')
    best_val_epoch = np.argmax(np.array(valid_curve))
    # best_train = max(train_curve)
    print('Finished training!')
    print('Best validation score: {}'.format(valid_curve[best_val_epoch]))
    print('Test score: {}'.format(test_curve[best_val_epoch]))
    if not args['eval_results_path'] == '':
        result_dict = {'Val': valid_curve, 'Test': test_curve}
        torch.save(result_dict, args['eval_results_path'])




In [ ]:
dataset.eval_metric

'F1'

# Run

In [7]:
 main()

Starting Training
{'device': 0, 'num_layer': 3, 'emb_dim': 256, 'drop_ratio': 0.5, 'batch_size': 64, 'lr': 0.001, 'epochs': 50, 'num_vocab': 5000, 'max_seq_len': 5, 'diff_pool_layers': [(32, 5)], 'max_num_nodes': 1000, 'random_split': False, 'dataset': 'ogbg-code2', 'num_workers': 0, 'model_save_path': 'best_model_params', 'eval_results_path': 'eval_results'}
Target seqence less or equal to 5 is 0.9874166466036873%.


/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


tensor([     0,      1,      2,  ..., 407973, 407974, 407975])
tensor([407976, 407977, 407978,  ..., 430790, 430791, 430792])
tensor([430793, 430794, 430795,  ..., 452738, 452739, 452740])
407976


/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Coverage of top 5000 vocabulary:
0.9025832389087423
       attr idx      attr
0             0       NaN
1             1       NaN
2             2        \t
3             3        \n
4             4      \n\t
...         ...       ...
10025     10025         |
10026     10026         }
10027     10027         ~
10028     10028  __NONE__
10029     10029   __UNK__

[10030 rows x 2 columns]
model size: 37.513MB
#Params: 9828562
=====Epoch 1
Training...


...

Iteration:   0%|          | 0/357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/343 [00:00<?, ?it/s]

{'Validation': {'precision': 0.15493564739740837, 'recall': 0.09179487794884211, 'F1': 0.1094309380850118}, 'Test': {'precision': 0.16151433691756273, 'recall': 0.09801225696523673, 'F1': 0.11595004761352491}}
=====Epoch 34
Training...


Iteration:   0%|          | 0/6375 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Some random exploration

In [ ]:
import networkx as nx
import torch_geometric

In [ ]:
print(len(train_loader))

In [ ]:
print(len(valid_loader))

In [ ]:
print(len(test_loader))

In [ ]:
dataset[0]

In [ ]:
G = torch_geometric.utils.to_networkx(dataset[0])
nx.draw(G, with_labels = True)



In [ ]:
print(G.number_of_edges(), G.number_of_nodes())
G.number_of_edges() / G.number_of_nodes()

In [ ]:
dataset[0]

In [ ]:
for d in dataset[:1]:
  for pair in zip(d.x, d.node_is_attributed):
    print(pair)
